# 라이브러리 불러오기

In [1]:
import openai
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

openai.api_key  = os.getenv('OPENAI_API_KEY')

# OpenAIEmbeddings - ada-002

In [4]:
from langchain.embeddings import OpenAIEmbeddings

embeddings_model = OpenAIEmbeddings(openai_api_key=openai.api_key)

In [5]:
# !pip install tiktoken

In [6]:
embeddings = embeddings_model.embed_documents(
    [
        "안녕하세요",
        "제 이름은 홍길동 입니다",
        "이름이 무엇인가요?",
        "랭체인은 유용합니다",
        "Hello World"
    ]
)

In [9]:
len(embeddings), len(embeddings[0])

(5, 1536)

In [10]:
embeddings[0]

[-0.013645178495461919,
 -0.00945306251426354,
 -0.0060033666644602095,
 -0.02298943831233707,
 -0.012281941028265285,
 0.01790769687943346,
 -0.01984055082395007,
 0.005875362888441581,
 -0.012819555397427317,
 0.008691440838576939,
 0.012051534603960806,
 -0.0033024916936464344,
 -0.023936664205965134,
 -0.014374798994313206,
 -0.005510553104677252,
 -0.00724500133606338,
 0.027546364542961093,
 -0.013273968196933737,
 0.007769815607020337,
 -0.017536486580905283,
 -0.004265718408522813,
 -0.015898040483041154,
 0.0031200865689336127,
 -0.012032333525330566,
 1.440665043011318e-06,
 0.003996910758280482,
 0.0038945081099946315,
 -0.023962264402375283,
 0.014221195021884429,
 -0.021555797511044642,
 0.002766476871098723,
 0.006777787972690571,
 -0.018778120321185823,
 -0.005433750652801549,
 0.009958676637912885,
 -0.012518747501672301,
 0.0022352620853779144,
 -0.01482281174471709,
 0.00528654672947531,
 -0.007225800723094454,
 0.018227706785141348,
 -0.014810010715189387,
 0.0091010

In [11]:
embedded_query_q = embeddings_model.embed_query("이 대화에서 언급된 이름은 무엇입니까")
embedded_query_a = embeddings_model.embed_query("이 대화에서 언급된 이름은 홍길동 입니다.")
print(len(embedded_query_q), len(embedded_query_a))

1536 1536


In [7]:
import numpy as np
from numpy.linalg import norm

def cos_sim(A, B):
    return np.dot(A, B)/(norm(A)*norm(B))

In [13]:
print(cos_sim(embedded_query_q, embedded_query_a))
print(cos_sim(embedded_query_q, embeddings[1]))
print(cos_sim(embedded_query_q, embeddings[3]))

0.9115427318561372
0.8560436628549855
0.7749074108300489


# Huggingface Embedding

In [19]:
# !pip install sentence_transformers

In [8]:
from langchain.embeddings import HuggingFaceBgeEmbeddings

In [9]:
model_name = "BAAI/bge-small-en"
# model_kwargs = {"device":"cpu"}
model_kwargs = {"device":"cuda:0"}
encode_kwargs = {"normalize_embeddings":True}
hf = HuggingFaceBgeEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)

In [10]:
embeddings = hf.embed_documents(
    [
        "today is monday",
        "weather is nice today",
        "what's the problem?",
        "langchain is useful",
        "Hello World",
        "my anme is tom"
    ]
)

In [13]:
BGE_query_q = hf.embed_query("hello? who is this?")
BGE_query_s = hf.embed_query("hi this is harrison")

In [14]:
print(cos_sim(BGE_query_q, BGE_query_s))
print(cos_sim(BGE_query_q, embeddings[1]))
print(cos_sim(BGE_query_q, embeddings[5]))

0.852254019231775
0.746906805363989
0.7897432433217707


In [15]:
sentences = [
    "안녕하세요",
    "제 이름은 홍길동 입니다",
    "이름이 무엇인가요",
    "랭체인은 유용합니다",
    "홍길동 아버지의 이름은 홍상직 입니다."
]
ko_embeddings = hf.embed_documents(sentences)

In [16]:
BGE_query_q_2 = hf.embed_query("홍길동은 아버지를 아버지라 부르지 못하였습니다. 홍길동의 아머지의 이름은 무엇입니까?")
BGE_query_s_2 = hf.embed_query("홍길동의 아버지는 엄했습니다.")

In [19]:
print("홍길동의 아버지는 엄했습니다.", cos_sim(BGE_query_q, BGE_query_s))
print(sentences[1], cos_sim(BGE_query_q_2, ko_embeddings[1]))
print(sentences[3], cos_sim(BGE_query_q_2, ko_embeddings[3]))
print(sentences[4], cos_sim(BGE_query_q_2, ko_embeddings[4]))

홍길동의 아버지는 엄했습니다. 0.852254019231775
제 이름은 홍길동 입니다 0.8841384190703682
랭체인은 유용합니다 0.8468634929057983
홍길동 아버지의 이름은 홍상직 입니다. 0.9152941189254451


# 한국어 사전학습 모델 임베딩 - ko-sbert-nli

In [20]:
from langchain.embeddings import HuggingFaceBgeEmbeddings

In [22]:
model_name = "jhgan/ko-sbert-nli"
model_kwargs = {'device':'cuda:0'}
encode_kwargs = {'normalize_embeddings':True}
ko = HuggingFaceBgeEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)

In [23]:
sentences = [
    "안녕하세요",
    "제 이름은 홍길동 입니다",
    "이름이 무엇인가요",
    "랭체인은 유용합니다",
    "홍길동 아버지의 이름은 홍상직 입니다."
]
ko_embeddings = ko.embed_documents(sentences)

In [24]:
ko_query_q = ko.embed_query("홍길동은 아버지를 아버지라 부르지 못하였습니다. 홍길동의 아머지의 이름은 무엇입니까?")
ko_query_s = ko.embed_query("홍길동의 아버지는 엄했습니다.")

In [25]:
print("홍길동의 아버지는 엄했습니다.", cos_sim(ko_query_q, ko_query_s))
print(sentences[1], cos_sim(ko_query_q, ko_embeddings[1]))
print(sentences[3], cos_sim(ko_query_q, ko_embeddings[3]))
print(sentences[4], cos_sim(ko_query_q, ko_embeddings[4]))

홍길동의 아버지는 엄했습니다. 0.8038877310865097
제 이름은 홍길동 입니다 0.5625218360547132
랭체인은 유용합니다 0.24756831074293642
홍길동 아버지의 이름은 홍상직 입니다. 0.6217444149311113
